<a href="https://colab.research.google.com/github/as9786/NLP/blob/main/Seq2seqAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention 신경망 구현 및 학습

In [273]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [274]:
import random
import tensorflow as tf
from konlpy.tag import Okt
import pandas as pd

# 초매개변수

In [275]:
EPOCHS = 200
NUM_WORDS = 2000

# Encoder

In [276]:
class Encoder(tf.keras.Model):

  def __init__(self):
    super(Encoder,self).__init__()
    self.emb = tf.keras.layers.Embedding(NUM_WORDS,64) 
    self.lstm = tf.keras.layers.LSTM(512,return_state=True)

  def call(self,x,training=False,mask=None):
    x = self.emb(x)
    _,h,c = self.lstm(x)
    return h,c

# Decoder

In [277]:
class Decoder(tf.keras.Model):

  def __init__(self):
    super(Decoder,self).__init__()
    self.emb = tf.keras.layers.Embedding(NUM_WORDS,64)
    self.lstm = tf.keras.layers.LSTM(512,return_sequences=True,return_state=True) 
    self.dense = tf.keras.layers.Dense(NUM_WORDS,activation='softmax')

  def call(self,inputs,training=False,mask=None): 
    x, h, c = inputs  # h와 c는 context
    x = self.emb(x) 
    x, h, c = self.lstm(x,initial_state=[h,c])
    return self.dense(x),h,c

# Seq2seq

In [278]:
class Seq2seq(tf.keras.Model):

  def __init__(self,sos,eos):
    super(Seq2seq,self).__init__()
    self.enc = Encoder()
    self.dec = Decoder()
    self.sos = sos
    self.eos = eos
  
  def call(self, inputs, training=False, mask=None): 
    if training is True:
      x,y = inputs # 학습을 위해서는 정답 data도 같이 알아야 함
      h,c = self.enc(x) # Hidden state, cell state
      y, _, _ = self.dec((y,h,c))
      return y #전체 문장장

    else:
      x = inputs # Test 시에는 정답이 있으면 안됨됨
      h,c = self.enc(x)
      y = tf.convert_to_tensor(self.sos) # 첫 번째 입력
      y = tf.reshape(y,(1,1))

      seq = tf.TensorArray(tf.int32,64)

      for idx in tf.range(64):
        y, h, c = self.dec([y,h,c])
        y = tf.cast(tf.argmax(y,axis=-1),dtype=tf.int32)
        y = tf.reshape(y,(1,1))
        seq = seq.write(idx,y)

        if y == self.eos:
          break
      return tf.reshape(seq.stack(),(1,64))

# 학습, test loop

In [279]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

In [280]:
dataset_file = '/content/drive/MyDrive/chatbot_data.csv' # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

# 학습 환경 정의

In [281]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [282]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [283]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# 학습

In [284]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 3.2846128940582275, Accuracy: 83.26676177978516
Epoch 2, Loss: 0.6036694645881653, Accuracy: 91.03618621826172
Epoch 3, Loss: 0.5606583952903748, Accuracy: 91.04010009765625
Epoch 4, Loss: 0.5551127791404724, Accuracy: 91.12625122070312
Epoch 5, Loss: 0.5385661721229553, Accuracy: 91.08708953857422
Epoch 6, Loss: 0.5362086892127991, Accuracy: 91.09492492675781
Epoch 7, Loss: 0.5332222580909729, Accuracy: 91.114501953125
Epoch 8, Loss: 0.5413007140159607, Accuracy: 91.19282531738281
Epoch 9, Loss: 0.5195093750953674, Accuracy: 91.15757751464844
Epoch 10, Loss: 0.5135628581047058, Accuracy: 91.16541290283203
Epoch 11, Loss: 0.5076462030410767, Accuracy: 91.29463958740234
Epoch 12, Loss: 0.5013461709022522, Accuracy: 91.35337829589844
Epoch 13, Loss: 0.4787765145301819, Accuracy: 91.54918670654297
Epoch 14, Loss: 0.4643762409687042, Accuracy: 91.92903900146484
Epoch 15, Loss: 0.45136940479278564, Accuracy: 92.14051055908203
Epoch 16, Loss: 0.44590917229652405, Accuracy: 92.

In [285]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q: ', test_text)
    print('a: ', gt_text)
    print('p: ', texts)

_
q:  ['여기 기프티콘 되죠 \n']
a:  ['\t 네 현금영수증 해드릴까 요 \n']
p:  ['쿠폰 10 개 를 모으면 무료 로 아이스 아메리카노 를 제공 하고 있어요 \n']
_
q:  ['네 에 테이크 아웃 도 가능한가요 \n']
a:  ['\t 네 로 오시 면 테이크 아웃 잔 에 담아 드려요 \n']
p:  ['아뇨 캐리어 는 2 개 이상 부터 가능해요 \n']
_
q:  ['아메리카노 톨 사이즈 로 주세요 \n']
a:  ['\t 따뜻한 거 로 드릴 까요 \n']
p:  ['네 사이즈 는 어떤 걸 로 주문 넣어 드릴 까요 \n']
_
q:  ['진동 을 따로 주시나요 \n']
a:  ['\t 주 번호 로 드리겠습니다 \n']
p:  ['네 담아 드립니다 \n']
_
q:  ['자리 있나요 \n']
a:  ['\t 네 있습니다 \n']
p:  ['네 자리 있습니다']
_
q:  ['그럼 루이보스 밀크 티 하나 \n']
a:  ['\t 네 알겠습니다 \n']
p:  ['어니언 크림 이랑 플레인 크림 있습니다 \n']
_
q:  ['다음 에 무료 로 하고 엔 도장 찍어주세요 \n']
a:  ['\t 네 \n']
p:  ['네 따로 추가 하실 건 없으신 가요 \n']
_
q:  ['아메리카노 한 잔 에 얼마 죠 \n']
a:  ['\t 입니다 \n']
p:  ['아메리카노 와 주스 가 있습니다 \n']
_
q:  ['얼마나 \n']
a:  ['\t 바로 만들어 드릴게요 \n']
p:  ['6700원 결제 도 와 드리겠습니다 \n']
_
q:  ['카푸치노 는 로 주시 고 아메리카노 는 로 \n']
a:  ['\t 네 더 없으세요 \n']
p:  ['드시고 가실 건가 요 \n']
_
q:  ['아메리카노 는 어떤 종류 가 있나요 \n']
a:  ['\t 디카 페인 과 기본 아메리카노 2 종류 있습니다 \n']
p:  ['네 또 더 필요한 거 있으세요 \n']
_
q:  ['카카오 페이 로 결제 가능한가요 \n']
a:  ['\t 네 가능합니다 \n']